# RQ4: Replace Tokenizer

## Prepare Dataset

Usually, the default WordLevel tokenizer treats each node value as token. The dataset has to be prepared such that the tokenizer can identify between tokens. Therefore, a Pre-Tokenizer can be used which splits up text into words. The final tokens will be parts of those words. As a delimiter, `,` is selected. This means that every `,` has to be removed from strings.

In [2]:
import json
from tqdm import tqdm

# Clean up raw py150 dataset by removing "," character

if False:
    with open("data/python150k.json") as fin, open("data/python150k_rq4.json", "w") as fout:
        for i, line in enumerate(tqdm(fin)):
            dp = json.loads(line.strip())
            for j, d in enumerate(dp):
                if "value" in d:
                    if "," in d["value"]:
                        d["value"].replace(",", " ")
            print(json.dumps(dp), file=fout)
    
    with open("data/python150k_rq4.json") as fin:
        for line in tqdm(fin):
            dp = json.loads(line.strip())
            for d in enumerate(dp):
                if "value" in d:
                    if "," in d["value"]:
                        print('Not cleaned up')


In [4]:
if False:
    with open("output/test_new_trees.json") as fin, open("output/test_raw.json", "w") as fout:
        for i, line in enumerate(tqdm(fin)):
            dp = json.loads(line)
            raw = ",".join([d["value"] for d in dp if "value" in d])
            print(json.dumps(raw), file=fout)

## Train Tokenizer

In [2]:
from tokenizers import Tokenizer
from tokenizers.models import WordPiece
from tokenizers.trainers import WordPieceTrainer
from tokenizers.pre_tokenizers import CharDelimiterSplit

tokenizer = Tokenizer(WordPiece(unk_token="[UNK]"))
tokenizer.pre_tokenizer = CharDelimiterSplit(delimiter=",")
trainer = WordPieceTrainer(special_tokens=["[UNK]", "[PAD]"])

# tokenizer.train(["output/train_raw.json"], trainer)

# tokenizer.save("output/tokenizer.json")


## Test Tokenizer

In [9]:
tokenizer = Tokenizer.from_file("output/tokenizer.json")
encoding = tokenizer.encode("The,quick,brown,fox,jumped,over,the,lazy,dog").ids
print(encoding)
for e in encoding:
    print("{}: {}".format(e, tokenizer.decode([e])))
print(tokenizer.get_vocab_size())
print(tokenizer.decode([214]))

[1404, 109, 21495, 3980, 819, 5089, 144, 17913, 267, 3348, 12577, 9618, 21341]
1404: Th
109: ##e
21495: quick
3980: br
819: ##own
5089: fo
144: ##x
17913: jump
267: ##ed
3348: over
12577: the
9618: lazy
21341: dog
30000
Str


## New Tokenizer

Using the ``new_trees.json`` trees, iterate through each tree individually
Tokenize each ``type`` or ``value`` and count the resulting tokens up until ``max_len`` and split there
Combine with ``get_dfs`` to receive a custom ``dps.txt`` file